Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [1]:
# data from https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease
# read in the data
DATA_PATH = 'https://raw.githubusercontent.com/reggieHabas/DS-Unit-2-Applied-Modeling/master/module1-define-ml-problems/kidney_disease.csv'
!pip install category_encoders==2.*

     |████████████████████████████████| 81kB 2.6MB/s 


In [3]:
import pandas as pd
pd.options.display.max_columns = None
kidney = pd.read_csv('https://raw.githubusercontent.com/reggieHabas/DS-Unit-2-Applied-Modeling/master/module1-define-ml-problems/kidney_disease.csv')

In [4]:
kidney.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35,7300,4.6,no,no,no,good,no,no,ckd


In [ ]:
# The majority class occurs with 62% frequency
kidney['classification'].value_counts(normalize=True)

ckd       0.625
notckd    0.375
Name: classification, dtype: float64

Evaluation metric: accuracy

For the train val test split: I'll use a random split considering there is no time series data.

In [5]:
kidney.isna().sum().sort_values()

id                  0
classification      0
pe                  1
appet               1
ane                 1
cad                 2
dm                  2
htn                 2
pcc                 4
ba                  4
age                 9
bp                 12
sc                 17
bu                 19
bgr                44
al                 46
sg                 47
su                 49
hemo               52
pc                 65
pcv                70
sod                87
pot                88
wc                105
rc                130
rbc               152
dtype: int64

In [6]:
kidney['classification'].describe()

count     400
unique      2
top       ckd
freq      250
Name: classification, dtype: object

In [7]:
kidney.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [8]:
# fill missing values using sklearn imputer
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(strategy='most_frequent')
kidney = my_imputer.fit_transform(kidney)

In [10]:
kidney_disease = pd.DataFrame(data=kidney, columns=['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'])

In [11]:
kidney_disease.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48,80,1.02,1,0,normal,normal,notpresent,notpresent,121,36,1.2,135,3.5,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7,50,1.02,4,0,normal,normal,notpresent,notpresent,99,18,0.8,135,3.5,11.3,38,6000,5.2,no,no,no,good,no,no,ckd
2,2,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,53,1.8,135,3.5,9.6,31,7500,5.2,no,yes,no,poor,no,yes,ckd
3,3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,56,3.8,111,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,26,1.4,135,3.5,11.6,35,7300,4.6,no,no,no,good,no,no,ckd
